In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!ls -lha kaggle.json

In [9]:
#!pip install -q kaggle

In [10]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/

In [11]:
#!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#!pwd

In [13]:
#!kaggle datasets list

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          13294        273  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51          10089        211  1.0              
ambaliyagati/spotify-dataset-for-playing-around-with-sql            Spotify dataset                                     302KB  2024-06-17 10:48:20           1978         43  1.0              
ihelon/coffee-sales                     

In [14]:
#!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
100% 1.06G/1.06G [00:57<00:00, 23.4MB/s]
100% 1.06G/1.06G [00:57<00:00, 20.0MB/s]


In [15]:
#!unzip amazon-books-reviews.zip

Archive:  amazon-books-reviews.zip
  inflating: Books_rating.csv        
  inflating: books_data.csv          


## **File Reading Approaches**

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **Pandas:**

In [12]:
df_ratings = pd.read_csv('Books_rating.csv')
df_books = pd.read_csv('books_data.csv')

> Pandas took almost ~1 minute to load the data.

In [ ]:
# Deleting the dataframes

lst = [df_ratings, df_books]
del lst
del df_ratings, df_books

### **Files:**

In [14]:
with open('Books_rating.csv', 'r') as f:
  df_ratings = f.read()

with open('books_data.csv', 'r') as f:
  df_books = f.read()

> Reading by open() & read() function takes ~ 15 seconds to read the files. Which is an improvement over the Pandas library.

In [17]:
with open('Books_rating.csv', 'r') as f:
  df_ratings = f.readlines()

with open('books_data.csv', 'r') as f:
  df_books = f.readlines()

> Reading by open() & readlines() function takes ~ 5 seconds to read the files. Which is much faster compared to read() function and Pandas library.